<a href="https://colab.research.google.com/github/MauricioTellezNava/tumor_detection_KNN/blob/main/cancer_data/tumor_detection_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge: Cancer data

## Objetivo:
Detectar los tumores malignos (M) y beningnos (B) de la variable diagnosis mediante el dataset que recaba múltiples características

1. Aplicar el algoritmo de KNN a [data.csv](https://drive.google.com/file/d/1CaTF1C-iNnz7dPKaUcKRMK4BC19E0QLT/view?usp=sharing) (No olvides estandarizar y crear una variable nueva a partir de las existentes)
2. Evaluar el rendimiento del modelo con el reporte de la clasificación

## Importando las bibliotecas

In [ ]:
import numpy as np # Numeric Python, me deja manipular facilmente arreglos y listas numéricos
import pandas as pd # Pandas es la principal biblioteca de manejo de bases de datos en Python
import matplotlib.pyplot as plt # Este es para hacer grafiquitas
import seaborn as sns # Está basado en matplotlib, y hace gráficas más bonitas más fácil.

-%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importando la base de datos

In [ ]:

url = '/content/drive/MyDrive/DEV F/Master/data.csv'
df = pd.read_csv(url)

In [ ]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [ ]:
df["diagnosis"].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

# Sklearn

## Paso 3: Separo mi base en atributos (variables independientes, si quieren verlo así) y etiquetas (variable dependiente)

In [ ]:
# Set de características
X = df.drop(columns = ['diagnosis', 'Unnamed: 32'])



In [ ]:
# Columna de respuestas
y = df.iloc[:, 1]
y

0      M
1      M
2      M
3      M
4      M
      ..
564    M
565    M
566    M
567    M
568    B
Name: diagnosis, Length: 569, dtype: object

El algoritmo k-nearest neighbors (KNN) es un algoritmo de aprendizaje automático simple y supervisado que puede utilizarse para resolver tanto problemas de clasificación como de regresión. Es fácil de implementar y entender, pero tiene el gran inconveniente de volverse significativamente lento a medida que crece el tamaño de los datos en uso. KNN funciona encontrando las distancias entre una consulta y todos los ejemplos de los datos, seleccionando el número especificado de ejemplos (K) más cercanos a la consulta, y luego vota por la etiqueta más frecuente (en el caso de la clasificación) o promedia las etiquetas (en el caso de la regresión). En el caso de la clasificación y la regresión, la elección de la K correcta para nuestros datos se realiza probando varias K y eligiendo la que mejor funciona.

En la clasificación k-NN, el resultado es la pertenencia a una clase. Un objeto se clasifica mediante un voto plural de sus vecinos, asignando el objeto a la clase más común entre sus k vecinos más cercanos (k es un número entero positivo, normalmente pequeño). Si k = 1, el objeto se asigna simplemente a la clase de ese único vecino más cercano.

## Paso 4: Vamos a dividir nuestra base en una base de entrenamiento y otra de prueba. Dejemos 80% para que el KNN memorice, y 20% para ver qué tan bien "aprendió" a clasificar

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True,test_size = 0.20)

In [ ]:
print(len(df))
print(len(x_train))

569
455


In [ ]:
import math
math.sqrt(455)

21.330729007701542

## Paso 5: Normalizo mis datos para que tengan una escala común

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)

MinMaxScaler()

Ahora sí, re-escalo mis variables

In [ ]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_test

array([[0.00999004, 0.09940631, 0.62529591, ..., 0.1676197 , 0.14902425,
        0.11196379],
       [0.00094318, 0.24733821, 0.43151843, ..., 0.32142611, 0.24916223,
        0.16810967],
       [0.00095392, 0.48137972, 0.32499155, ..., 0.66792973, 0.26867731,
        0.15499147],
       ...,
       [0.00097292, 0.21102988, 0.27798444, ..., 0.09631416, 0.11945594,
        0.10055096],
       [0.00987762, 0.23703449, 0.31653703, ..., 0.3191526 , 0.10959984,
        0.1104552 ],
       [0.00098192, 0.38079584, 0.34697328, ..., 0.73062349, 0.31973191,
        0.30735931]])

In [ ]:
x_train.min(), x_train.max()

(0.0, 1.0000000000000002)

## Paso 6: Importo mi clasificador y le digo que memorice los datos de entrenamiento

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 21)
classifier.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=21)

In [ ]:
x_train[:11]

array([[8.69735224e-05, 2.32618615e-01, 3.00642543e-01, 2.35951060e-01,
        1.30635290e-01, 4.24483163e-01, 2.95938634e-01, 1.87558575e-01,
        1.89910537e-01, 4.36868687e-01, 2.90016849e-01, 1.03059931e-01,
        1.51122702e-01, 8.19865241e-02, 4.52738831e-02, 1.78128293e-01,
        1.26610989e-01, 6.82070707e-02, 1.96438719e-01, 1.92046069e-01,
        9.29756920e-02, 2.48530994e-01, 3.77957472e-01, 2.25875792e-01,
        1.45631392e-01, 4.71703097e-01, 2.04237855e-01, 2.13338658e-01,
        3.49638305e-01, 2.85629805e-01, 2.12908304e-01],
       [9.90565434e-04, 3.52828615e-01, 1.76530267e-01, 3.45212690e-01,
        2.18989742e-01, 2.55935723e-01, 2.26388604e-01, 1.29779756e-01,
        1.60089463e-01, 3.16666667e-01, 1.41743892e-01, 4.81984429e-02,
        1.61775106e-02, 5.03698817e-02, 3.18197063e-02, 5.29625727e-02,
        1.16021270e-01, 5.83333333e-02, 1.59102103e-01, 6.95292572e-02,
        5.12762047e-02, 3.39981793e-01, 1.85684337e-01, 3.18538314e-01,
       

## Paso 7: Hago la predicción con mis atributos del set de prueba

In [ ]:
x_test[0]

array([0.00999004, 0.09940631, 0.62529591, 0.09788021, 0.0483148 ,
       0.24564413, 0.09204849, 0.0361059 , 0.05183897, 0.28333333,
       0.20134794, 0.02411733, 0.29322843, 0.02289968, 0.00863149,
       0.1657205 , 0.07846907, 0.03661616, 0.11562796, 0.15185128,
       0.04744137, 0.07059505, 0.74573226, 0.06309185, 0.0359984 ,
       0.26302582, 0.08111884, 0.05742812, 0.1676197 , 0.14902425,
       0.11196379])

In [ ]:
y_pred = classifier.predict(x_test)
y_pred

array(['B', 'B', 'M', 'M', 'M', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B',
       'B', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'M', 'M', 'B', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
       'B', 'B', 'B', 'B', 'M', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'M', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B',
       'B', 'B', 'M', 'B', 'M', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'B',
       'B', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'M', 'M', 'M', 'B', 'B', 'B', 'B', 'M'], dtype=object)

## Paso 8: revisamos qué tan bien lo hicimos

In [ ]:
y_test.values, y_pred

(array(['B', 'M', 'M', 'M', 'M', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'M',
        'B', 'M', 'M', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
        'M', 'M', 'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'M', 'B',
        'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
        'B', 'B', 'B', 'B', 'M', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'B',
        'M', 'B', 'M', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B',
        'B', 'B', 'M', 'B', 'M', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'M',
        'B', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
        'B', 'B', 'M', 'M', 'M', 'B', 'B', 'B', 'B', 'M'], dtype=object),
 array(['B', 'B', 'M', 'M', 'M', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B',
        'B', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
        'M', 'M', 'B', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'B',
        'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
        'B', 'B', 'B', 'B', 'M', 'M', 'M', 'B', 'B

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[69  0]
 [ 6 39]]
              precision    recall  f1-score   support

           B       0.92      1.00      0.96        69
           M       1.00      0.87      0.93        45

    accuracy                           0.95       114
   macro avg       0.96      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114



In [ ]:
error = []
for i in range(1, 119):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))
    
error

[0.043859649122807015,
 0.07017543859649122,
 0.05263157894736842,
 0.05263157894736842,
 0.043859649122807015,
 0.043859649122807015,
 0.043859649122807015,
 0.05263157894736842,
 0.043859649122807015,
 0.043859649122807015,
 0.043859649122807015,
 0.05263157894736842,
 0.043859649122807015,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0.05263157894736842,
 0

## Práctica

In [ ]:
len(y_pred)

114

In [ ]:
prediction = pd.DataFrame(y_pred)

In [ ]:
prediction = prediction.rename(columns={0:"expected_result"})

In [ ]:
prediction["expected_result"]

0      B
1      B
2      M
3      M
4      M
      ..
109    B
110    B
111    B
112    B
113    M
Name: expected_result, Length: 114, dtype: object